# Solving LP problems with Xpress library

https://www.msi-jp.com/xpress/learning/square/01-python-interface.pdf

Andrea Gasparin: andrea.gasparin@PHD.units.it

In [ ]:
# install xpress (the solvere we will use)
!pip install xpress
# import xpress (usually as xp)
import xpress as xp
import numpy as np

     |████████████████████████████████| 65.6 MB 25 kB/s 
Using the Community license in this session. If you have a full Xpress license, first set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPAUTH_PATH environment variable to: /usr/local/lib/python3.7/dist-packages/xpress/license/community-xpauth.xpr
NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.



#The assignment problem

A company has 4 machines available for the assignment to 4 tasks. Any
machine can be assigned to any task, but only one, and each task requires to be processed
by one machine only. The cost required to set up each machine for the
processing of each task is given in the table below.

 .|Task 1 |Task 2 |Task 3 |Task 4
-------|-------|-------|-------|------
Machine 1|13| 4| 7| 6
Machine 2| 1| 11 |5 |4
Machine 3| 6 |7 |2 |8
Machine 4| 1| 3| 5| 9

The company wants to minimise the total setup cost needed for the
processing of all four tasks.

It is intuitive to assign the decision variables as:

$$
x_{ij} = \begin{cases} &1 \quad \text{if machine $i$ is assigned to process task $j$}\\
&0 \quad \text{if machine $i$ is not assigned to process task $j$}
\end{cases}
$$
with $i = 1, 2, 3, 4$ and $j = 1, 2, 3, 4$.

The problem can be then formulated as follow:


$
\begin{gather}
\min 13x_{11} + 4x_{12} + 7x_{13} + 6x_{14}+
x_{21} + 11x_{22} + 5x_{23} + 4x_{24}+
6x_{31} + 7x_{32} + 2x_{33} + 8x_{34}+
x_{41} + 3x_{42} + 5x_{43} + 9x_{44}\\
x_{11} + x_{12} + x_{13} + x_{14} \leq 1\\
x_{21} + x_{22} + x_{23} + x_{24} \leq 1\\
x_{31} + x_{32} + x_{33} + x_{34} \leq 1\\
x_{41} + x_{42} + x_{43} + x_{44} \leq 1\\
x_{11} + x_{21} + x_{31} + x_{41} = 1\\
x_{12} + x_{22} + x_{32} + x_{42} = 1\\
x_{13} + x_{23} + x_{33} + x_{43} = 1\\
x_{14} + x_{24} + x_{34} + x_{44} = 1\\
x_{ij} \in \{0, 1\},\quad for\quad i = 1, . . . , 4,\; j = 1, . . . , 4
\end{gather}
$

Equivalently, defining 

$$
c_{ij} = \text{cost of machine $i$ to perform task $j$}\\
I = \{1,2,3,4\}\\
J = \{1,2,3,4\}
$$

we can rewrite the problem in a more compact way as:

$
\begin{gather}
\min \sum_{i\in I,j\in J} c_{ij}x_{ij}\\
∑_{j\in J} x_{ij}  \leq 1 \quad \forall i \in I\\
∑_{i\in I} x_{ij}  = 1 \quad \forall j \in J\\
x_{ij} \in \{0, 1\},\quad for\quad i \in I,\; j \in J
\end{gather}
$

In [ ]:
assignment = xp.problem()
assignment.setControl('outputlog', 0) #this is used to quite the xpress outputs (no printing)

X = np.array([[xp.var(vartype=xp.binary) for i in range(4)]for j in range(4)], dtype=xp.npvar)
assignment.addVariable(X)

I = range(4)
J = range(4)

costs = np.array([[13,	4,	7, 6],
                	[1,	11,	5, 4],
                  [6,	7,	2, 8],
                  [1,	3,	5, 9]])


for i in I:
  assignment.addConstraint( xp.Sum(X[i,j] for j in J) <= 1)

for j in J:
  assignment.addConstraint( xp.Sum(X[i,j] for i in I) == 1 )

assignment.setObjective( 
    xp.Sum( xp.Sum(costs[i,j]*X[i,j]   for j in J )  for i in I)
    )   #sense=minimise is the default value

assignment.solve()

X_sol = assignment.getSolution(X)
print("\n\n Solution:\n",X_sol, "\n")

for i in I:
  for j in J:
    if X_sol[i,j] > 0.9: #==1
      print("Machine ", i, " is assigned to task ", j)



 Solution:
 [[-0.  1. -0. -0.]
 [-0. -0. -0.  1.]
 [-0. -0.  1. -0.]
 [ 1. -0. -0. -0.]] 

Machine  0  is assigned to task  1
Machine  1  is assigned to task  3
Machine  2  is assigned to task  2
Machine  3  is assigned to task  0


#Set covering


A typical application concerns facility location. Suppose we are given a
set of potential sites $N = \{1, . . . , n\}$ for the location of fire stations. A
station placed at $j$ costs $c_j$ . We are also given a set of communities
$M = \{1, . . . , m\}$ that have to be protected. The subset of communities
that can be protected from a station located at $j$ is $M_j$ . For example,
$M_j$ might be the set of communities that can be reached from $j$ within 10
minutes. Then the problem of choosing a minimum-cost set of locations
for the fire stations such that each community can be reached from some
fire station in 10 minutes is a set-covering problem.

### Decision variables

$$
x_{j} = \begin{cases} &1 \quad \text{if} \; \text{a station is placed in $j$}\\
&0 \quad \text{otherwise}
\end{cases}
$$

### Obj

$$
\min \;\sum_{j=1}^n c_j x_{j}
$$

###Constraints 
Let A be the $m \times n$ incidence matrix of the family $\{M_j\}$ for $j \in N$, that is, for $i \in M$,

$$
a_{ij} =\begin{cases} &1 \quad \text{if}\; i \in M_j\\
&0 \quad \text{otherwise}
\end{cases}
$$
A solution $x \in \{0, 1\}^n$ covers all sites if and only if  satisfies
$$
Ax ≥ \overline{1}
$$
where $\overline{1}$ is a $m$−vector all of whose components are equal to 1.


###In this case
Let 
$$
M = \{1, 2, 3, 4, 5\}, \\
\{M\} = \{\{1, 3, 5\}, \{2, 3\}, \{1, 4\}, \{3, 4, 5\}, \{2\}, \{5\}, \{1, 5\}\},\\
c = [3, 5, 1, 9, 2, 4, 1].
$$
 Hence,
$$
A=
\begin{pmatrix}
1 &0 &1 &0 &0 &0 &1\\
0 &1 &0 &0 &1 &0 &0\\
1 &1 &0 &1 &0 &0 &0\\
0 &0 &1 &1 &0 &0 &0\\
1 &0 &0 &1 &0 &1 &1\\
\end{pmatrix}
$$

In [ ]:
set_covering= xp.problem()
set_covering.setControl('outputlog', 0)

costs = np.array([3,5,1,9,2,4,1])

A = np.array([[1, 0, 1, 0, 0, 0, 1],
              [0, 1, 0, 0, 1, 0, 0],
              [1, 1, 0, 1, 0, 0, 0],
              [0, 0, 1, 1, 0, 0, 0],
              [1, 0, 0, 1, 0, 1, 1]])
b = np.ones(5)
x = np.array([xp.var(vartype=xp.binary) for _ in range(7)], dtype=xp.npvar)

set_covering.addVariable(x)
set_covering.addConstraint(xp.Dot(A,x) >= b )
set_covering.setObjective(xp.Dot(costs, x) , sense= xp.minimize)

set_covering.solve()

x_sol = set_covering.getSolution(x)
print("\n\n",x_sol)




 [1. 0. 1. 0. 1. 0. 0.]



## Parameter tuning

In [ ]:
print(set_covering.getControl())
print(type(set_covering.getControl()))

{'extrarows': 0, 'extracols': 0, 'extraelems': 0, 'lpiterlimit': 2147483647, 'ifmsg': 0, 'lplog': 100, 'scaling': 163, 'presolve': 1, 'crash': 2, 'pricingalg': 0, 'invertfreq': -1, 'invertmin': 3, 'maxnode': 2147483647, 'maxtime': 0, 'maxmipsol': 0, 'siftpasses': 4, 'defaultalg': 1, 'varselection': -1, 'nodeselection': 4, 'backtrack': 3, 'miplog': -100, 'keepnrows': -1, 'mpsecho': 0, 'maxpagelines': 23, 'outputlog': 0, 'extrapresolve': 0, 'barsolution': 0, 'cachesize': -1, 'crossover': -1, 'bariterlimit': 500, 'choleskyalg': -1, 'baroutput': 1, 'extramipents': 0, 'refactor': -1, 'barthreads': -1, 'keepbasis': 1, 'omniformat': 0, 'ifecho': 0, 'crossoverops': 0, 'version': 3901, 'crossoverthreads': -1, 'bigmmethod': 1, 'elimfillin': 10, 'nppass': -1, 'nrpass': -1, 'presolveops': 511, 'mippresolve': -1, 'mipthreads': -1, 'barorder': 0, 'breadthfirst': 11, 'iquiet': 0, 'autoperturb': 1, 'densecollimit': 0, 'prtcut': 0, 'callbackfrommasterthread': 0, 'maxmcoeffbufferelems': 2147483647, 'ref

In [ ]:
print(set_covering.getControl('maxnode'), "\n")

set_covering.setControl('maxnode', 100)
print(set_covering.getControl('maxnode'), "\n")

set_covering.setControl({'maxnode': 20, 'presolve':0})
print(set_covering.getControl('maxnode'))
print(type(set_covering.getControl()))

2147483647 

100 

20
<class 'dict'>


# Try your self

#1

Get the knapsack problem, just riveseted in the following way

Item  | Weight  | Rating | Availability
------|---------|--------|--------
Ant Repellent |1      | 2    | 4
Beer          | 7     |  9   |2
Blanket       | 4     | 3    |3
Bratwurst     |3 |8|2
Brownies      | 5| 10|1
Frisbee       | 6| 6| 1
Salad         |2| 2|3
Watermelon    |7| 10| 1

#2
Same problem but now with 2 bags at disposal
